In [1]:
import pandas as pd
import matplotlib as plt
from sqlalchemy import create_engine
import datetime

## 유저의 월별 광고비

In [14]:
start = '2016-01-01 00:00:00'
end = '2019-12-31 23:59:59'
q = f'''
select date_format(created_at, '%%Y-%%m') as monthly, uid, sum(pay_point) as pay_point
from vw_ad_view
where created_at between '{start}' and '{end}' and pay_point != 0
group by date_format(created_at, '%%Y-%%m'), uid
'''
pay_point = pd.read_sql(q, con=mysql)

In [15]:
pay_point

,monthly,uid,pay_point
0,2016-01,6445,995400.0
1,2016-01,11254,1131600.0
2,2016-01,12893,77200.0
3,2016-01,19692,82000.0
4,2016-01,21251,85800.0
...,...,...,...
132321,2019-12,10490341,8920.0
132322,2019-12,10501657,45230.0
132323,2019-12,10504562,9700.0
132324,2019-12,10508193,2940.0


In [18]:
uid_cnt = pay_point.groupby('monthly', as_index=False)['uid'].count()
uid_cnt.head()

,monthly,uid
0,2016-01,778
1,2016-02,657
2,2016-03,778
3,2016-04,1711
4,2016-05,1953


In [30]:
point_df = pd.DataFrame(columns=['monthly', 'points_range', 'uid_cnt', 'uid_percentage', 'points_sum', 'points_percentage'])
month_list = uid_cnt['monthly'].to_list()
for m in month_list:
    month = pay_point[pay_point['monthly'] == m]
    all_points = month['pay_point'].sum()
    one = month[(month['pay_point'] < 10000)]
    two = month[(month['pay_point'] >= 10000) & (month['pay_point'] < 50000)]
    three = month[(month['pay_point'] >= 50000) & (month['pay_point'] < 100000)]
    four = month[(month['pay_point'] >= 100000) & (month['pay_point'] < 500000)]
    five = month[(month['pay_point'] >= 500000)]
    uids = uid_cnt[uid_cnt['monthly'] == m]['uid'].values[0]
    point_df = point_df.append({'monthly':m, 'points_range': '0~1', 'uid_cnt': len(one), 'uid_percentage': len(one)/uids, 'points_sum': one['pay_point'].sum(), 'points_percentage': one['pay_point'].sum()/all_points}, ignore_index=True)
    point_df = point_df.append({'monthly':m, 'points_range': '1~5', 'uid_cnt': len(two), 'uid_percentage': len(two)/uids, 'points_sum': two['pay_point'].sum(), 'points_percentage': two['pay_point'].sum()/all_points}, ignore_index=True)
    point_df = point_df.append({'monthly':m, 'points_range': '5~10', 'uid_cnt': len(three), 'uid_percentage': len(three)/uids, 'points_sum': three['pay_point'].sum(), 'points_percentage': three['pay_point'].sum()/all_points}, ignore_index=True)
    point_df = point_df.append({'monthly':m, 'points_range': '10~50', 'uid_cnt': len(four), 'uid_percentage': len(four)/uids, 'points_sum': four['pay_point'].sum(), 'points_percentage': four['pay_point'].sum()/all_points}, ignore_index=True)
    point_df = point_df.append({'monthly':m, 'points_range': '50~', 'uid_cnt': len(five), 'uid_percentage': len(five)/uids, 'points_sum': five['pay_point'].sum(), 'points_percentage': five['pay_point'].sum()/all_points}, ignore_index=True)


In [31]:
point_df

,monthly,points_range,uid_cnt,uid_percentage,points_sum,points_percentage
0,2016-01,0~1,194,0.249357,738200.0,0.009160
1,2016-01,1~5,276,0.354756,8384700.0,0.104043
2,2016-01,5~10,149,0.191517,11352400.0,0.140868
3,2016-01,10~50,123,0.158098,26749900.0,0.331930
4,2016-01,50~,36,0.046272,33363800.0,0.413999
...,...,...,...,...,...,...
235,2019-12,0~1,731,0.183254,2897816.0,0.004333
236,2019-12,1~5,1602,0.401604,47503362.0,0.071029
237,2019-12,5~10,615,0.154174,43740888.0,0.065403
238,2019-12,10~50,773,0.193783,173820343.0,0.259903


# 유저의 월별 거래액

In [32]:
q = f'''
select pay.monthly as monthly, pay.uid as uid, pay.pay_point as pay_point, product.avg_price as avg_price
from (select date_format(created_at, '%%Y-%%m') as monthly, uid, sum(pay_point) as pay_point
	from vw_ad_view
	where created_at between '{start}' and '{end}' and pay_point != 0
	group by date_format(created_at, '%%Y-%%m'), uid) as pay
left outer join (select date_format(create_date, '%%Y-%%m') as monthly, uid, avg(price) as avg_price
                from product_info
                where create_date between '{start}' and '{end}'
                group by date_format(create_date, '%%Y-%%m'), uid) as product on pay.monthly = product.monthly and pay.uid = product.uid'''
avg_price = pd.read_sql(q, con=mysql)

In [33]:
avg_price

,monthly,uid,pay_point,avg_price
0,2016-01,6445,995400.0,10540.9945
1,2016-01,11254,1131600.0,15557.2519
2,2016-01,12893,77200.0,NaN
3,2016-01,19692,82000.0,403691.7492
4,2016-01,21251,85800.0,20000.0000
...,...,...,...,...
132321,2019-12,10490341,8920.0,4400.0000
132322,2019-12,10501657,45230.0,500.0000
132323,2019-12,10504562,9700.0,414250.0000
132324,2019-12,10508193,2940.0,4325.0000


In [36]:
q = f'''
select monthly, uid, sum(cnt) as sum
from 
(
    (select TO_CHAR(om.create_date, 'YYYY-mm') as monthly, oi.seller_id as uid, count(om.id) as cnt
    from order_mast om
    join order_item oi on oi.order_mast_id = om.id
    where om.create_date BETWEEN '{start}' AND '{end}' and om.order_status_cd = 'purchase_confirm'
    group by TO_CHAR(om.create_date, 'YYYY-mm'), oi.seller_id)
    union all
    (select TO_CHAR(created_at, 'YYYY-mm') as monthly, uid, count(*) as cnt
    from wire_transfer
    where created_at BETWEEN '{start}' AND '{end}' and status = 'transfer_completed'
    group by TO_CHAR(created_at, 'YYYY-mm'), uid)
    union all 
    (select TO_CHAR(created_at, 'YYYY-mm') as monthly, seller_uid as uid, count(*) as cnt
    from bunjang_promise
    where created_at BETWEEN '{start}' AND '{end}' and status = 4
    group by TO_CHAR(created_at, 'YYYY-mm'), seller_uid) 
) as trans
group by monthly, uid'''
bun = pd.read_sql(q, con=redshift)
bun

,monthly,uid,sum
0,2018-03,6823915,16
1,2018-03,4036716,2
2,2018-03,6683225,1
3,2018-03,3650845,1
4,2018-03,2917757,2
...,...,...,...
4783915,2019-12,4698896,1
4783916,2019-12,8604734,1
4783917,2019-12,7715288,1
4783918,2019-12,3931779,1


In [42]:
trans_df = pd.DataFrame(columns=['monthly', 'trans_range', 'uid_cnt', 'uid_percentage', 'points_sum', 'points_percentage'])

for m in month_list:
    m1 = avg_price[avg_price['monthly'] == m]
    m2 = bun[bun['monthly'] == m]
    month = pd.merge(m1, m2, on = 'uid')
    month['trans'] = month['avg_price'] * month['sum']
    month = month[['monthly_x', 'uid', 'pay_point', 'trans']]
    
    all_points = month['pay_point'].sum()
    
    one = month[(month['trans'] < 100000)]
    two = month[(month['trans'] >= 100000) & (month['trans'] < 500000)]
    three = month[(month['trans'] >= 500000) & (month['trans'] < 1000000)]
    four = month[(month['trans'] >= 1000000) & (month['trans'] < 5000000)]
    five = month[(month['trans'] >= 5000000)]
    
    uids = len(month)
    if uids != 0:
        trans_df = trans_df.append({'monthly':m, 'trans_range': '0~10', 'uid_cnt': len(one), 'uid_percentage': len(one)/uids, 'points_sum': one['pay_point'].sum(), 'points_percentage': one['pay_point'].sum()/all_points}, ignore_index=True)
        trans_df = trans_df.append({'monthly':m, 'trans_range': '10~50', 'uid_cnt': len(two), 'uid_percentage': len(two)/uids, 'points_sum': two['pay_point'].sum(), 'points_percentage': two['pay_point'].sum()/all_points}, ignore_index=True)
        trans_df = trans_df.append({'monthly':m, 'trans_range': '50~100', 'uid_cnt': len(three), 'uid_percentage': len(three)/uids, 'points_sum': three['pay_point'].sum(), 'points_percentage': three['pay_point'].sum()/all_points}, ignore_index=True)
        trans_df = trans_df.append({'monthly':m, 'trans_range': '100~500', 'uid_cnt': len(four), 'uid_percentage': len(four)/uids, 'points_sum': four['pay_point'].sum(), 'points_percentage': four['pay_point'].sum()/all_points}, ignore_index=True)
        trans_df = trans_df.append({'monthly':m, 'trans_range': '500~', 'uid_cnt': len(five), 'uid_percentage': len(five)/uids, 'points_sum': five['pay_point'].sum(), 'points_percentage': five['pay_point'].sum()/all_points}, ignore_index=True)


In [43]:
trans_df

,monthly,trans_range,uid_cnt,uid_percentage,points_sum,points_percentage
0,2016-10,0~10,179,0.555901,31650270.0,0.407937
1,2016-10,10~50,94,0.291925,25265050.0,0.325639
2,2016-10,50~100,13,0.040373,6724000.0,0.086665
3,2016-10,100~500,12,0.037267,5268600.0,0.067906
4,2016-10,500~,4,0.012422,613650.0,0.007909
...,...,...,...,...,...,...
190,2019-12,0~10,297,0.098901,23234148.0,0.037139
191,2019-12,10~50,694,0.231102,49433473.0,0.079017
192,2019-12,50~100,434,0.144522,35292274.0,0.056413
193,2019-12,100~500,936,0.311688,176681039.0,0.282416
